<a href="https://colab.research.google.com/github/onyekaokonji/Skin-Cancer-Detection/blob/main/Skin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Input, Flatten
from tensorflow.keras.initializers import RandomNormal
import numpy as np
from sklearn.metrics import *
import matplotlib.pyplot as plt

## Loading dataset

In [ ]:
train_dir = '/content/drive/MyDrive/data/train'
test_dir = "/content/drive/MyDrive/data/valid"
valid_dir = "/content/drive/MyDrive/data/test"

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir, labels='inferred', image_size=(160, 160), label_mode='categorical', batch_size=64)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir, labels='inferred', image_size=(160, 160), label_mode='categorical')
valid_data = tf.keras.preprocessing.image_dataset_from_directory(directory=valid_dir, labels='inferred', image_size=(160, 160), label_mode='categorical', batch_size=10)

In [ ]:
train_class_names = train_data.class_names
print(train_class_names)

valid_class_names = valid_data.class_names
print(valid_class_names)

test_class_names = test_data.class_names
print(test_class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(scale= 1./255., offset=0.0)
normalized_train_data = train_data.map(lambda x, y: (normalization_layer(x), y))

rotation_layer = tf.keras.layers.RandomRotation(factor=0.3, fill_mode='reflect', interpolation='bilinear', seed=None, fill_value=0.0)
rotated_train_data = train_data.map(lambda x, y: (rotation_layer(x), y))

flip_layer = tf.keras.layers.RandomFlip(mode='horizontal_and_vertical', seed=None)
flipped_train_data = train_data.map(lambda x,y: (flip_layer(x), y))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
print(train_data)
print(valid_data)

## Model construction

In [ ]:
input = Input(shape = (160, 160, 3))
conv1 = Conv2D(filters=16, kernel_size=(3,3), strides=1, padding='same', activation='relu', kernel_initializer='random_normal')(input)
conv2 = Conv2D(filters=16, kernel_size=(1,1), strides=1, padding='same', activation='relu', kernel_initializer='random_normal')(conv1)
conv3 = Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', activation='relu', kernel_initializer='random_normal')(conv2)
conv4 = Conv2D(filters=32, kernel_size=(1,1), strides=1, padding='same', activation='relu', kernel_initializer='random_normal')(conv3)
pool1 = MaxPooling2D(pool_size=(2,2))(conv4)
conv5 = Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu', kernel_initializer='random_normal')(pool1)
conv6 = Conv2D(filters=64, kernel_size=(1,1), strides=1, padding='same', activation='relu', kernel_initializer='random_normal')(conv5)
flatten = Flatten()(conv6)
fc1 = Dense(units = 100, activation='relu', kernel_initializer='random_normal')(flatten)
dropout1 = Dropout(rate=0.3)(fc1)
fc2 = Dense(units= 50, activation='relu', kernel_initializer='random_normal')(dropout1)
output = Dense(units=3, activation='softmax', kernel_initializer='random_normal')(fc2)

model = Model(inputs=input, outputs =output)
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss= 'categorical_crossentropy', metrics=['Accuracy'])

In [ ]:
model.fit(x=train_data, epochs= 50, validation_data=valid_data, validation_steps=10)

In [ ]:
# Obtaining predicted and true labels

y_true = []
y_pred = []

for image_batch, label_batch in test_data:
  y_true.append(label_batch)
  pred = model.predict(image_batch)
  y_pred.append(pred)

true_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

In [ ]:
true_labels = np.argmax(true_labels, axis= 1)
print(true_labels)

In [ ]:
predicted_labels = np.argmax(predicted_labels, axis=1)
print(predicted_labels)

In [ ]:
cm = confusion_matrix(true_labels, predicted_labels, labels = [0,1,2])
cm

In [ ]:
# Displaying Confusion Matrix
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


plt.figure(figsize=(12,8))
plot_confusion_matrix(cm, classes=['Melanoma', 'Nevus', 'Seborrheic Keratosis'], title='Confusion matrix, normalization')

In [ ]:
print(classification_report(predicted_labels, true_labels, target_names=['Melanoma', 'Nevus', 'Seborrheic Keratosis']))

In [ ]:
accuracy = accuracy_score(predicted_labels, true_labels, normalize=True)
print(accuracy)